In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import datasets
import lightning as L

from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader

from src.model.modeling_bind import LitBIND

/home/jjw1214/.conda/envs/jjw1214_py312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def train_valid_test_split(
    ds,
    split_ratio,
    seed: int,
):
    ds_train_test = ds.train_test_split(
        test_size=split_ratio[2] / sum(split_ratio), seed=seed
    )
    ds_train_valid = ds_train_test["train"].train_test_split(
        test_size=split_ratio[1] / (split_ratio[1] + split_ratio[0]), seed=seed
    )
    ds_train = ds_train_valid["train"]
    ds_valid = ds_train_valid["test"]
    ds_test = ds_train_test["test"]
    return ds_train, ds_valid, ds_test

In [4]:
SEED=42
DATASET_NAME = 'AutoML/bitabuse'
MINI_BATCH_SIZE=4
N_BATCH = 4
BASE_MODEL_NAME='google/gemma-3-1b-pt'
EPOCHS=20
LEARNING_RATE = 5e-5
USE_BNTD=True
MAX_LENGTH=32
SPLITS = (1, 20, 79)
INFERENCE_SENTENCE_MAX_LENGTH=256
INFERENCE_SENTENCE_MIN_LENGTH=128
INFERENCE_SENTENCE_N_OVERLAP=1

In [5]:
L.seed_everything(SEED)

Seed set to 42


42

In [8]:
ds = datasets.load_dataset(DATASET_NAME, split="train")
def preprocessing(example):
    example['sentence_noisy'] = example['text'][:MAX_LENGTH]
    example['sentence'] = example['label'][:MAX_LENGTH]
    return example
ds = ds.map(preprocessing)
ds_train, ds_valid, ds_test = train_valid_test_split(
    ds, SPLITS, SEED
)
ds_valid = ds_valid.select(range(len(ds_train)))
dl_train, dl_valid, dl_test = DataLoader(ds_train, batch_size=MINI_BATCH_SIZE), DataLoader(ds_valid, batch_size=MINI_BATCH_SIZE), DataLoader(ds_test, batch_size=MINI_BATCH_SIZE)

In [7]:
lit_bind = LitBIND(
    base_model_name=BASE_MODEL_NAME,
    lr=LEARNING_RATE,
    epochs=EPOCHS,
    use_bntd=USE_BNTD,
    inference_sentence_max_length=INFERENCE_SENTENCE_MAX_LENGTH,
    inference_sentence_min_length=INFERENCE_SENTENCE_MIN_LENGTH,
    inference_sentence_n_overlap=INFERENCE_SENTENCE_N_OVERLAP
)

The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


use full attn gemma3


In [8]:
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints/bind',
    filename=f"{DATASET_NAME.split('/')[1]}/{BASE_MODEL_NAME.split('/')[1]}"+"-{epoch:02d}-{valid_loss:.4f}",
    every_n_epochs=1,
    save_top_k=-1,
)

In [9]:
trainer = L.Trainer(
    callbacks=[checkpoint_callback],
    precision='16-mixed',
    max_epochs=EPOCHS,
    enable_checkpointing=True,
    accumulate_grad_batches=N_BATCH
)

Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(lit_bind, dl_train, dl_valid)

You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/home/jjw1214/.conda/envs/jjw1214_py312/lib/python3.12/site-packages/lightning/pytorch/utilities/model_summary/model_summary.py:231: Precision 16-mixed is not supported by the model summary.  Estimated model size in MB will not be accurate. Using 32 bits instead.

  | Name | Type | Params | Mode 
--------------------------------------
0 | bind | BIND | 999 M  | train
--------------------------------------
999 M     Trainable params
0         Non-trainable params
999 M     Total params
3,999.544 Total estimated model params size (MB)
1         Modules in train mode
450       Modules in eval mode


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00,  5.55it/s]

/home/jjw1214/.conda/envs/jjw1214_py312/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


/home/jjw1214/.conda/envs/jjw1214_py312/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Epoch 13:  47%|████▋     | 381/814 [00:53<01:00,  7.17it/s, v_num=1]

In [7]:
lit_bind = LitBIND.load_from_checkpoint(
    'checkpoints/bind/bitabuse/gemma-3-1b-pt-epoch=02-valid_loss=0.0189.ckpt',
    base_model_name=BASE_MODEL_NAME,
    lr=LEARNING_RATE,
    epochs=EPOCHS,
    use_bntd=USE_BNTD,
    inference_sentence_max_length=INFERENCE_SENTENCE_MAX_LENGTH,
    inference_sentence_min_length=INFERENCE_SENTENCE_MIN_LENGTH,
    inference_sentence_n_overlap=INFERENCE_SENTENCE_N_OVERLAP
)

The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


use full attn gemma3


In [10]:
trainer = L.Trainer()

Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [11]:
preds = trainer.predict(lit_bind, dl_test)

You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/home/jjw1214/.conda/envs/jjw1214_py312/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 64303/64303 [2:20:05<00:00,  7.65it/s]  


In [12]:
prediction = []
for pred in preds:
    prediction.extend(pred)

In [13]:
import pandas as pd
df = pd.DataFrame()
df['true'] = ds_test['sentence'][:]
df['pred'] = prediction
df.to_csv('./results/bind_bituse_gemma3.csv', index=False)

In [14]:
df

,true,pred
0,here's one of your pass word [re,here's one of your pass mord [re
1,"immediately after that, i will g","immediately after that, i will g"
2,"claims to be elon musk, just wai","claims to be alon musk, just wai"
3,in case this does not occur - ev,in case this does not occur - ev
4,this is insane they even know my,this is insane they even know my
...,...,...
257204,"you have good taste, etc","you have dood taste, etc"
257205,searched ip address originates f,searched ip address originates f
257206,well the previous time you visit,well the previous time you vlsit
257207,tinder iepn scam,tinder hepn scam
